# ACTIVIDAD #

### Construye un detector de movimiento en una región de interés de la imagen marcada manualmente. Guarda 2 ó 3 segundos de la secuencia detectada en un archivo de vídeo. ###

### Opcional: muestra el objeto seleccionado anulando el fondo. ###